In [7]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

def plotShit(filename,label):
    file = open(filename + '.txt')
    dets = file.readline()
    dat = file.readline()
    dat = dat.replace('[','')
    dat = dat.replace(']','')
    dat = dat.split(',')
    dat = map(int,dat)

    kek = np.array(dat)
    kek = kek.reshape((20,200))
    wtf = np.mean(kek,axis=0)
    return plt.plot(wtf,label=label,lw=2)
fig = plt.figure(num=None, figsize=(12,8), dpi=200)
a, = plotShit('exp01','AIXI')
b, = plotShit('exp02','Thompson Sampling')
c, = plotShit('exp03','Square KSA')
d, = plotShit('exp04','Shannon KSA')
e, = plotShit('exp05','Q-Learning')
plt.legend(handles=[c,d,a,b,e],loc=2,fontsize=15)
plt.xlabel('Cycles',fontsize=15)
plt.ylabel('Total reward',fontsize=15)
plt.title('7x7 Grid, Performance averaged over 20 runs',fontsize=20)
plt.savefig('../../../figures/agent-comparison-dispenser.png',bbox_inches='tight')
plt.close(fig)